In [198]:
import pandas as pd
import numpy as np

In [199]:
# Results as store visits
P_Campaign=pd.read_csv("/home/jian/Projects/Big_Lots/Facebook/P_Big-Lots-MS-Ad-Sets-Feb-25-2018-Mar-10-2018.csv",dtype=str)
S_Campaign=pd.read_csv("/home/jian/Projects/Big_Lots/Facebook/S_Big-Lots-MS-Ad-Sets-Feb-25-2018-Mar-10-2018.csv",dtype=str)

In [200]:
Start_Date=P_Campaign['Reporting Starts'].unique()[0]
End_Date=P_Campaign['Reporting Ends'].unique()[0]

In [201]:
def clean_df(df):
    df=df[['Reporting Starts','Reporting Ends','Ad Set Name','Results','Cost per Results','Amount Spent (USD)']]
    df=df.rename(columns={"Reporting Starts":"Start_Time","Reporting Ends":"End_Time","Ad Set Name":"Ad_Set_Name","Results":"Visits","Amount Spent (USD)":"Cost","Cost per Results":"CPSV"})
    df['Visits']=df['Visits'].astype(int)
    df['Cost']=df['Cost'].astype(float)
    df['CPSV']=df['CPSV'].astype(float)
    df=df[df['Visits']>0]
    df['Ad_Set_Name_Layer']=df['Ad_Set_Name'].apply(lambda x: len(x.split("_")))
    df=df[df['Ad_Set_Name_Layer']==5]
    df['Zip_Type']=df['Ad_Set_Name'].apply(lambda x: x.split("_")[0])
    df['Product']=df['Ad_Set_Name'].apply(lambda x: x.split("_")[1])
    df['Family_Status']=df['Ad_Set_Name'].apply(lambda x: x.split("_")[2])
    df['Retailer']=df['Ad_Set_Name'].apply(lambda x: x.split("_")[3])
    df['Income']=df['Ad_Set_Name'].apply(lambda x: x.split("_")[4])
    df.reset_index(inplace=True)
    del df['index']
    
    df['Std_Visits']=df['Visits'].apply(lambda x: (x-df['Visits'].mean())/df['Visits'].std())
    df['Std_CPSV']=df['CPSV'].apply(lambda x: (x-df['CPSV'].mean())/df['CPSV'].std())
    df['Score']=df['Std_Visits']*1.5-df['Std_CPSV']
    df=df.sort_values('Score', ascending=False)
    
    del df['Ad_Set_Name_Layer']
    return df

In [202]:
P_Campaign_output=clean_df(P_Campaign)
S_Campaign_output=clean_df(S_Campaign)

In [203]:
writer=pd.ExcelWriter('/home/jian/Projects/Big_Lots/Facebook/Optimized_from'+Start_Date+"_to_"+End_Date+".xlsx", engine='xlsxwriter')
P_Campaign_output.to_excel(writer,"P_Zips",index=False)
S_Campaign_output.to_excel(writer,"S_Zips",index=False)

In [204]:
workbook  = writer.book
worksheet_P = writer.sheets['P_Zips']
worksheet_S = writer.sheets['S_Zips']
'''
format_money_4 = workbook.add_format({'num_format':'$#,##0.0000'})
format_money_2 = workbook.add_format({'num_format':'$#,##0.00'})

worksheet_P.set_column('E:E', None, format_money_4)
worksheet_S.set_column('E:E', None, format_money_4)
worksheet_P.set_column('F:F', None, format_money_2)
worksheet_S.set_column('F:F', None, format_money_2)
'''


"\nformat_money_4 = workbook.add_format({'num_format':'$#,##0.0000'})\nformat_money_2 = workbook.add_format({'num_format':'$#,##0.00'})\n\nworksheet_P.set_column('E:E', None, format_money_4)\nworksheet_S.set_column('E:E', None, format_money_4)\nworksheet_P.set_column('F:F', None, format_money_2)\nworksheet_S.set_column('F:F', None, format_money_2)\n"

In [205]:
format_color_top = workbook.add_format()
format_color_top.set_bg_color('#ABEBC6')

format_color_bottom = workbook.add_format()
format_color_bottom.set_bg_color('#FDEBD0')

for i in range(14):
    worksheet_P.set_row(row=i,cell_format=format_color_top)
    worksheet_S.set_row(row=i,cell_format=format_color_top)
for i in range(6):
    worksheet_P.set_row(row=len(P_Campaign_output)-i,cell_format=format_color_bottom)
    worksheet_S.set_row(row=len(P_Campaign_output)-i,cell_format=format_color_bottom)
    
writer.save()